In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.image as mpimg 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import IPython.display
import codecs
import json
import cv2

In [ ]:
path='../input/leapgestrecog/leapGestRecog'
folders=os.listdir(path)
folders=set(folders)
different_classes=os.listdir(path+'/'+'00')
different_classes=set(different_classes)
print("The different classes that exist in this dataset are:")
print(different_classes,sep='\n')

In [ ]:
x=[]
z=[]
y=[]
threshold=200

for i in folders:
    print('***',i,'***')
    subject=path+'/'+i
    subdir=os.listdir(subject)
    subdir=set(subdir)
    for j in subdir:
        print(j)
        images=os.listdir(subject+'/'+j)
        for k in images:
            results=dict()
            results['y']=j.split('_')[0]
            img = cv2.imread(subject+'/'+j+'/'+k,0)
            img=cv2.resize(img,(int(160),int(60)))
            
            ret, imgf = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            imgD=np.asarray(img,dtype=np.float64)
            z.append(imgD)
            imgf=np.asarray(imgf,dtype=np.float64)
            x.append(imgf)
            y.append(int(j.split('_')[0]))
            results['x']=imgf

print(list(set(y)))

In [ ]:
l = []
list_names = []
for i in range(10):
    l.append(0)
for i in range(len(x)):
    if(l[y[i] - 1] == 0):
        l[y[i] - 1] = i
        if(len(np.unique(l)) == 10):
            break
for i in range(len(l)):
    %matplotlib inline
    print("Class Label: " + str(i + 1))
    plt.imshow(np.asarray(z[l[i]]), cmap  =cm.gray)
    plt.show()
    plt.imshow(np.asarray(x[l[i]]), cmap = cm.gray)     
    plt.show()

In [ ]:
x=np.array(x)
y=np.array(y)
y = y.reshape(len(x), 1)

In [6]:
x_data = x.reshape((len(x), 60, 160, 1))
x_data/=255
x_data=list(x_data)
for i in range(len(x_data)):
    x_data[i]=x_data[i].flatten()

## PCA to the data

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
x_data=np.array(x_data)
x_data=pca.fit_transform(x_data)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)  

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_further,y_train,y_further = train_test_split(x_data,y,test_size = 0.2)

## Scaling

In [9]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(x_train)
X_train = scaler.transform(x_train)  
X_test = scaler.transform(x_further)  

## Model accuracy

In [ ]:
from sklearn.metrics import accuracy_score
def model_accuracy(model):
    model.fit(X_train, y_train)
    y_pred_model = model.predict(X_test) 
    y_train_score_model=model.predict(X_train)
    print("accuracy of the model is:\nTest ", accuracy_score(y_further, y_pred_model, normalize=True, sample_weight=None))
    print('Train',accuracy_score(y_train, y_train_score_model, normalize=True, sample_weight=None))

## SGD classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd= SGDClassifier(loss='modified_huber',shuffle=True,random_state=101)
model_accuracy(sgd)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
model_accuracy(classifier) 


## Decision tree

In [15]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=10)
model_accuracy(clf) 

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rft = RandomForestClassifier(n_estimators=100, max_depth=15,random_state=0)
model_accuracy(clf_rft) 

## Logistic Regression

In [ ]:


from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver = 'lbfgs')
model_accuracy(logistic)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
model_accuracy(gnb)

## ANN

In [ ]:
from sklearn.neural_network import MLPClassifier
ann_clf = MLPClassifier()
model_accuracy(ann_clf)

## Gradient Descent Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gdc_model= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
model_accuracy(gdc_model)

## SVM

In [ ]:
from sklearn.svm import SVC 
svm_model_rbf = SVC(kernel = 'rbf', C = 10,probability=True).fit(X_train, y_train) 
model_accuracy(svm_model_rbf)